## Skupine aplikacij posameznih vhodnih primerov

In [ ]:
import os
import csv

def map_ids_to_groups(repo_path, apps_csv, groups_csv):
    """
    repo_path: mapa, kjer so datoteke (npr. generated_guis/)
    apps_csv: csv datoteka s stolpci [id;app_name]
    groups_csv: csv datoteka s stolpci [app_name;group_name]

    return: slovar {id: group_name}
    """

    # 1. Preberi imena datotek v repozitoriju -> seznam ID-jev
    ids = [name for name in os.listdir(repo_path) if os.path.isdir(os.path.join(repo_path, name))]

    # 2. Preberi aplikacije in zgradi slovar {id: app_name}
    id_to_app = {}
    with open(apps_csv, newline='', encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=',')
        next(reader)  # preskoči glavo
        for row in reader:
            id_to_app[row[0]] = row[1]

    # 3. Preberi skupine in zgradi slovar {app_name: group_name}
    app_to_group = {}
    with open(groups_csv, newline='', encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=',')
        next(reader)  # preskoči glavo
        for row in reader:
            app_to_group[row[0]] = row[2]

    # 4. Končni slovar {id: group}
    id_to_group = {}
    for id_ in ids:
        app_name = id_to_app.get(id_)
        if app_name:
            group = app_to_group.get(app_name, "NEZNANA_SKUPINA")
            id_to_group[id_] = group

    return id_to_group

In [ ]:
repo = "generated_guis"
apps = "rico_guis_details.csv"       # primer: "id;app_name"
groups = "rico_guis_app_details.csv"   # primer: "app_name;group_name"

result = map_ids_to_groups(repo, apps, groups)
# print(result)

business_ids = [id_ for id_, group in result.items() if group == "Business"]
print("ID-ji v skupini 'Business':", business_ids)

## Filtriranje in kopiranje datotek po skupinah

In [ ]:
import os
import csv

def map_bussiness_apps_ids_to_groups(apps_csv, groups_csv):
    """
    repo_path: mapa, kjer so datoteke (npr. generated_guis/)
    apps_csv: csv datoteka s stolpci [id;app_name]
    groups_csv: csv datoteka s stolpci [app_name;group_name]

    return: slovar {id: group_name}
    """

    # 1. Preberi aplikacije in zgradi slovar {id: app_name}
    id_to_app = {}
    with open(apps_csv, newline='', encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=',')
        next(reader)  # preskoči glavo
        for row in reader:
            id_to_app[row[0]] = row[1]

    # 2. Preberi skupine in zgradi slovar {app_name: group_name}
    app_to_group = {}
    apps = []
    with open(groups_csv, newline='', encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=',')
        next(reader)  # preskoči glavo
        for row in reader:
            app_to_group[row[0]] = row[2]
            apps.append(row[0]) if row[2] == "Business" else None

    print(sorted(apps))

    # 3. Končni slovar {id: group}
    id_to_group = {}
    for id_ in id_to_app:
        app_name = id_to_app.get(id_)
        if app_name:
            group = app_to_group.get(app_name, "NEZNANA_SKUPINA")
            id_to_group[id_] = group

    return id_to_group

In [ ]:
apps = "rico_guis_details.csv"       # primer: "id;app_name"
groups = "rico_guis_app_details.csv"   # primer: "app_name;group_name"

result = map_bussiness_apps_ids_to_groups(apps, groups)
# print(result)

business_ids = [id_ for id_, group in result.items() if group == "Business"]
print("ID-ji v skupini 'Business':", business_ids)
print("Število 'Business' aplikacij:", len(business_ids))

In [ ]:
import os
import shutil

def copy_files_by_ids(ids, source_dir, target_dir, extensions=(".png", ".jpg", ".jpeg", ".json")):
    """
    Kopira slike in json datoteke iz source_dir v target_dir glede na seznam ID-jev.
    Imena datotek v source_dir morajo ustrezati ID-jem (brez končnice).

    :param ids: seznam ID-jev (lahko števila, npr. [123, 456])
    :param source_dir: pot do izvorne mape
    :param target_dir: pot do ciljne mape
    :param extensions: tuple dovoljenih končnic
    """
    os.makedirs(target_dir, exist_ok=True)

    copied = []
    missing = []

    for id_raw in ids:
        id_str = str(id_raw)  # pretvori številko v niz
        found_any = False

        for ext in extensions:
            src_path = os.path.join(source_dir, id_str + ext)
            if os.path.exists(src_path):
                dst_path = os.path.join(target_dir, id_str + ext)
                shutil.copy2(src_path, dst_path)
                copied.append(id_str + ext)
                found_any = True
            else:
                missing.append(id_str + ext)  # manjkajoča datoteka s to končnico

    return copied, missing

In [ ]:
ids = business_ids
source = "C:\\Users\Marija\IdeaProjects\SERGUI\webapp\gui2rapp\staticfiles\\resources\combined"
target = "rico_guis_business"

copied, missing = copy_files_by_ids(ids, source, target)

print("Kopirane datoteke:", copied)
print("Manjkajoče datoteke:", missing)

In [ ]:
ids = [30982, 18784, 18782, 67044, 67045, 63575, 59370, 54377, 34346, 22151,
       34527, 67033, 9015, 49799, 49794, 53054, 9007]
source = "rico_guis_business"
target = "rico_guis_business_selected"

copied, missing = copy_files_by_ids(ids, source, target)

In [ ]:
ids = ['3261', '38961', '53054', '69587']
source = "rico_guis"
target = "rico_guis_business_selected"

copied, missing = copy_files_by_ids(ids, source, target)

## Merjenje kompleksnosti GUI-jev in selekcija najzahtevnejših vmesnikov


In [ ]:
import os
import json

def measure_complexity(gui_json):
    """
    Oceni kompleksnost GUI-ja glede na:
    - število vseh elementov
    - globino gnezdenosti
    """
    def count_elements(node, depth=1):
        if isinstance(node, dict):
            count = 1
            max_depth = depth
            for v in node.values():
                c, d = count_elements(v, depth + 1)
                count += c
                max_depth = max(max_depth, d)
            return count, max_depth
        elif isinstance(node, list):
            count = 0
            max_depth = depth
            for item in node:
                c, d = count_elements(item, depth + 1)
                count += c
                max_depth = max(max_depth, d)
            return count, max_depth
        else:
            return 0, depth

    total_elements, max_depth = count_elements(gui_json)
    # Primer: kompleksnost je kombinacija obeh
    return total_elements + max_depth


def select_most_complex_jsons(folder, top_n=5):
    """
    Iz mape prebere vse JSON datoteke in izbere `top_n` najbolj kompleksnih.
    Ne upošteva datotek, ki vsebujejo 'overlay' (v imenu ali vsebini).
    """
    results = []

    for fname in os.listdir(folder):
        if not fname.endswith(".json"):
            continue
        if "overlay" in fname.lower():
            continue

        fpath = os.path.join(folder, fname)
        try:
            with open(fpath, encoding="utf-8") as f:
                data = json.load(f)
            # preskoči, če JSON vsebuje overlay
            if isinstance(data, dict) and "overlay" in json.dumps(data).lower():
                continue

            complexity = measure_complexity(data)
            results.append((fname, complexity))
        except Exception as e:
            print(f"Napaka pri datoteki {fname}: {e}")

    # sortiraj po kompleksnosti (padajoče)
    results.sort(key=lambda x: x[1], reverse=True)
    return results[:top_n]

In [ ]:
folder = "rico_guis_business"
most_complex = select_most_complex_jsons(folder, top_n=40)

print("Najbolj kompleksni GUI-ji:")
for fname, score in most_complex:
    print(f"{fname}: kompleksnost = {score}")

print(most_complex)

In [5]:
import os

main_dir = "generated_guis"
repo_names = [name for name in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, name))]

print(repo_names)
print(len(repo_names))

['12740', '14283', '18782', '20947', '22151', '27360', '27382', '27707', '30982', '31390', '32310', '3261', '33383', '34346', '34517', '34527', '35526', '3727', '37505', '38961', '40673', '43872', '43977', '44756', '47926', '49794', '53054', '53469', '54377', '54468', '56905', '58124', '59429', '59576', '61851', '63575', '64858', '65592', '67044', '68368', '69574', '69587', '70410', '8640']
44
